## Check for GPU

In [1]:
!nvidia -smi L

/bin/bash: line 1: nvidia: command not found


## Get Helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

# Import series of helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-08-04 05:17:07--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-08-04 05:17:08 (91.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2025-08-04 05:17:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 64.233.170.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   698KB/s    in 0.8s    

2025-08-04 05:17:18 (698 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Becoming one with the data

In [4]:
import pandas as pd
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_data.count()

,0
id,7613
keyword,7552
location,5080
text,7613
target,7613


In [6]:
# Get the value counts for each column in the training data
print("Value counts for each column in train_data:")
for column in train_data.columns:
    print(f"\nColumn: {column}")
    display(train_data[column].value_counts())

Value counts for each column in train_data:

Column: id


,count
id,
10873,1
1,1
4,1
5,1
6,1
...,...
25,1
24,1
23,1



Column: keyword


,count
keyword,
fatalities,45
deluge,42
armageddon,42
damage,41
body%20bags,41
...,...
forest%20fire,19
epicentre,12
threat,11



Column: location


,count
location,
USA,104
New York,71
United States,50
London,45
Canada,29
...,...
"Silesia, Poland",1
"Hickville, USA",1
New York NYC,1



Column: text


,count
text,
11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...,10
He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam,6
The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.',6
#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade beÛ_,6
Madhya Pradesh Train Derailment: Village Youth Saved Many Lives,5
...,...
Fotoset: elanorofrohan: 10th December 2013 Green Carpet in Zurich for the Swiss Premiere of The Desolation... http://t.co/BQ3P7n7w06,1
Beautiful desolation. Just me a couple of coyotes some lizards and the morning sun. #Phoenix #Arizona http://t.co/K2tBES65oa,1
The Hobbit: The Desolation of Smaug - Ed Sheeran 'I See Fire' [HD] http://t.co/OXRwRJZmnu,1



Column: target


,count
target,
0,4342
1,3271


## Visualize the dataset

0 is for not disaster
1 is for disaster

In [7]:
train_data["text"][:5]

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [8]:
# Shuffle training dataframe
train_df_shuffled = train_data.sample(frac = 1,random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does the test dataframe look like ?
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class ?
train_df_shuffled["target"].value_counts()

,count
target,
0,4342
1,3271


In [11]:
len(train_data),len(test_data)

(7613, 3263)

In [12]:
# Visualize some randome training examples
import random
random_index = random.randint(0,len(train_data)-5) # Create random indexes not higher than length of data
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target:{target}","Real Disaster" if target > 0 else "No Real Disaster")
  print(f"Text\n{text}\n")
  print("------------------------\n")

Target:1 Real Disaster
Text
#news #crimes Police ID victim in fatal crash that closed I-65 near Franklin: One person is d... http://t.co/9h0ym9OFsv #justice #courts

------------------------

Target:1 Real Disaster
Text
@Bilsko and suddenly I'm inundated with research. @humofthecity

------------------------

Target:0 No Real Disaster
Text
I liked a @YouTube video http://t.co/M9YdA5k6jf Spyro 3 Texture Hacks - 'Desolate Gardens' [In-game]

------------------------

Target:1 Real Disaster
Text
NASA MODIS image: Dust storm over Morocco and the Straits of Gibraltar http://t.co/QWQnni7VMZ #duststorm

------------------------

Target:1 Real Disaster
Text
Typhoon Soudelor approaches after 7 killed 2 missing in floods in Philippines http://t.co/nJMiDySXoF via @abc7chicago

------------------------



## Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split
train_sentence,val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                        train_df_shuffled.target.to_numpy(),
                                                                        test_size = 0.1,
                                                                        random_state=42)

In [14]:
len(train_sentence),len(train_labels),len(val_labels),len(val_sentences)

(6851, 6851, 762, 762)

In [15]:
# Check
train_sentence[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers
When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.


There are a few ways to do this, 2 methods -
1. Tokenization -
    * direct mapping from word to number can be modelled but quickly gets too big.
    * disadvantage is if we one hot encode the numbers the arrays will be very large and inconvinient.
2. Embedding -
  * richer representation of relationships between tokens (can limit(i.e set its size with our own will) size + can be learned by the models)

## Text vectorization (Tokenization)

In [16]:
## Text vectorization (tokenization)
train_sentence[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens = None, # How many words in the vocab (automatically add <OOV> - meaning out of vocab)
                                    standardize = "lower_and_strip_punctuation", # lowers all chars and removes punctuation marks
                                    split = "whitespace", # splits all words by spaces
                                    ngrams = None, # Create group of n numbers as a token
                                    output_mode = "int", # int
                                    output_sequence_length = None, # how long do you want your sequence to be ?
                                    )

In [18]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split())for i in train_sentence])/len(train_sentence)) # we got our max output_sequence_length from here

15

In [19]:
# Set up text vectorization variables
max_vocab_length = 10000 # max no of words to have in our vocabulary
max_length = 15 # max lenght our sequence will be (eg how many words form a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [20]:
# Fit the tex vectorizer to the training text
text_vectorizer.adapt(train_sentence)

In [21]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street !"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentence)
print(f"Original:\n{random_sentence}")
print("________\n")
print(f"Vectorized:\n{text_vectorizer([random_sentence])}")

Original:
SpaceX Founder Musk: Structural Failure Took Down Falcon 9 http://t.co/LvIzO9CSSR
________

Vectorized:
[[8255    1    1  384  320 1166  134 3012  491    1    0    0    0    0
     0]]


In [23]:
# Get the unique words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary() # Get all of the unique words in the data
top_5_words = words_in_vocab[:5] # most common words
bottom_5_words = words_in_vocab[-5:] # get least
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")
## Here '[UNK]' is unknown it represents all the words outside the vocab (which is called OOV - OUT OF VOCAB)

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Bottom 5 least common words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


## Create a Embedding using a Embedding layer
Turns positive integers (indexes) into dense vectors of fixed size.


The parameters we care most about for out embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean the vector will be 100 long. (Use a number which is divisible by 8)
* `input_length` = lenght of the sequences being passed to the embedding layer

In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set input shape here its 15)\`
                             output_dim = 128,# Output shape
                             embeddings_initializer = "uniform",
                             input_length = max_length # How long is each input
                             )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentence)
print(f"Original sentence:\n{random_sentence}")
sample_embd = embedding(text_vectorizer([random_sentence]))
print(f"Embeddings :\n {sample_embd}")
# Turns positive integers (indexes) into dense vectors of fixed size
print(f"Embeddings shape: {sample_embd.shape}")

Original sentence:
U.S. in record hurricane drought http://t.co/8JvQI9UspL
Embeddings :
 [[[ 0.01675284  0.03197647  0.0294291  ... -0.03570664 -0.0488929
    0.01160916]
  [ 0.01360143 -0.03066292  0.01001249 ...  0.03865849  0.02652979
    0.01140608]
  [ 0.04769497 -0.01947534 -0.01835631 ... -0.00632859  0.01481112
    0.01498659]
  ...
  [ 0.01148399  0.03215823  0.02785804 ...  0.01412169  0.01933691
    0.04134599]
  [ 0.01148399  0.03215823  0.02785804 ...  0.01412169  0.01933691
    0.04134599]
  [ 0.01148399  0.03215823  0.02785804 ...  0.01412169  0.01933691
    0.04134599]]]
Embeddings shape: (1, 15, 128)


In [26]:
# Check out a single token's embedding
sample_embd[0][0], sample_embd[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01675284,  0.03197647,  0.0294291 ,  0.00698414,  0.03023804,
        -0.01511264,  0.03224837,  0.03503117, -0.04677754, -0.00069178,
        -0.04117426, -0.02009041, -0.04757954,  0.03982189,  0.02302975,
         0.02835572, -0.03538501, -0.0321336 , -0.04618671,  0.03908649,
         0.02590531, -0.04887426,  0.02632488,  0.01325352,  0.03122944,
        -0.04701832, -0.04243619, -0.02623585, -0.00787544, -0.03866682,
        -0.01390117,  0.02569385, -0.0426637 ,  0.04679785, -0.0477207 ,
         0.04127664,  0.00597525,  0.03904731, -0.02497118, -0.01111681,
         0.02109495,  0.04771588,  0.0139082 ,  0.02285925,  0.02527275,
         0.01447251, -0.0269676 , -0.02059388, -0.03592565,  0.02273795,
         0.03453765,  0.03346959,  0.04982687, -0.01314436, -0.02365844,
         0.04905403, -0.04057115,  0.0294604 ,  0.01096507,  0.03185395,
        -0.03693123,  0.04974612,  0.03742388,  0.01281859, -0.01977935,
  

## Modelling a text dataset (and running a series of experiments)
Now we have a got way to tuen our text sequence into numbers, it's time to start building a series of modelling experiements.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed - forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bi-directional-LSTM model (RNN)
* Model 5: 1-D Convulutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pre-trained Feature Extractor (Using Transfer Learning for NLP)
* Model 7: Same as model 6 with 10% of training data.

How are we going to approach all of these ?

Use the stardard steps in modellig with Tensorflow:
* Create a Model
* Build a model
* Fit a model
* Evaluate our model

## Model 0: Getting a Baseline

As with all the machine learning modelling experimenets, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create baseline, we will use Sklearn's Multinomial Naive Bayes using TF-IDF formula to convert our words to numbers.

> 🔑 `Note`: We use Non-DL  algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.



In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),# Convert words to numbers using tfidf
    ("clf",MultinomialNB())# model the text
]
)

# Fit the pipeline to the training data
model_0.fit(train_sentence,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [28]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [29]:
display(train_data.target.value_counts())

,count
target,
0,4342
1,3271


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [31]:
import sklearn
sklearn.metrics.accuracy_score(val_labels,baseline_preds)
sklearn.metrics.precision_score(val_labels,baseline_preds)
sklearn.metrics.recall_score(val_labels,baseline_preds)
sklearn.metrics.f1_score(val_labels,baseline_preds)
sklearn.metrics.confusion_matrix(val_labels,baseline_preds)

array([[386,  28],
       [130, 218]])

In [32]:
# Create a function to evaluate our models
def evaluate_result(y_true,y_preds):
  import sklearn
  acc = sklearn.metrics.accuracy_score(y_true,y_preds)
  prec = sklearn.metrics.precision_score(y_true,y_preds)
  recall = sklearn.metrics.recall_score(y_true,y_preds)
  f1 = sklearn.metrics.f1_score(y_true,y_preds)
  return {"accuracy":round(acc,2),
          "precision":round(prec,2),
          "recall":round(recall,2),
          "f1":round(f1,2)}

In [33]:
baseline_results = evaluate_result(val_labels,baseline_preds)

## Model 1: A simple dense model

In [34]:
# Create a tensorboard callback(need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

In [35]:
# Use TensorFlow's Functional API

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string) # Inputs are 1-Dimensional string

x = text_vectorizer(inputs) # Convert the words to integers

x = embedding(x) # Create an embedding of the numberized inputs

x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (Without it, the code produces errors)
# x = layers.MaxPool1D()(x) # Just checking (Produces error)
outputs = layers.Dense(1, activation = "sigmoid")(x) # Create the output layer, want binary output so use sigmoid activation

model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Compile the model
model_1.compile(
    loss = "BinaryCrossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [38]:
# Fit it
model_1_history = model_1.fit(
    train_sentence,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                             experiment_name = "model_1_dense")]

)

Saving TensorBoard log files to: model_logs/model_1_dense/20250804-051723
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6475 - loss: 0.6467 - val_accuracy: 0.7467 - val_loss: 0.5376
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8028 - loss: 0.4677 - val_accuracy: 0.7795 - val_loss: 0.4722
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8680 - loss: 0.3465 - val_accuracy: 0.7874 - val_loss: 0.4610
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8902 - loss: 0.2865 - val_accuracy: 0.7861 - val_loss: 0.4716
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9138 - loss: 0.2461 - val_accuracy: 0.7874 - val_loss: 0.4855


In [39]:
# Check results
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7810 - loss: 0.5212


[0.48554491996765137, 0.787401556968689]

In [40]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


(762, 1)

In [41]:
model_1_pred_probs[:10]

array([[0.29643536],
       [0.8037959 ],
       [0.9981822 ],
       [0.08012913],
       [0.11431746],
       [0.9372385 ],
       [0.9384823 ],
       [0.99366003],
       [0.9625656 ],
       [0.1938246 ]], dtype=float32)

In [42]:
# Convert model predictions probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [43]:
# Calculate our model_1 results
model_1_results = evaluate_result(val_labels,model_1_preds)
model_1_results

{'accuracy': 0.79, 'precision': 0.83, 'recall': 0.67, 'f1': 0.74}

In [44]:
# Compare the results
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True,  True])

## Visualize learned embeddings

In [45]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [46]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [47]:
# Get the weight matrix of embedding layer
# (THESE ARE THE NUMERICAL REPRESENTATION OF EACH TOKEN IN OUR TRAINING DATA, WHICH HAVE BEEN LEARNED FOR 5 EPOCHS)

embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape # same size as vocab size and embedding_dim

(10000, 128)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector : https://projector.tensorflow.org/

And tensorflow also has an incredible guide about embeddings.

In [48]:
embed_weights

array([[ 0.03089165,  0.05446863,  0.00712787, ...,  0.03600794,
         0.04093195,  0.06232388],
       [-0.00554239,  0.00905224, -0.03791152, ...,  0.04996619,
         0.01675065,  0.02894905],
       [ 0.0482712 , -0.00194301, -0.03210685, ...,  0.02678074,
         0.04872223, -0.02693079],
       ...,
       [ 0.04745897, -0.00859883, -0.00652341, ..., -0.04164541,
        -0.01436969,  0.03628366],
       [ 0.06535843,  0.04756736, -0.00110641, ...,  0.06002775,
         0.07806967,  0.06321669],
       [ 0.0269771 ,  0.01734787, -0.07977051, ...,  0.03018608,
         0.10470331,  0.09053259]], dtype=float32)

In [49]:
# Create embedding files (got this from Tensorflow's word embeddings documentation)
import io
out_v = io.open("vectors.tsv","w",encoding = "utf-8")
out_m = io.open("metadata.tsv","w",encoding = "utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0, it's padding and <UNK> token
  vec = embed_weights[index]
  out_v.write("\t".join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [50]:
from google.colab import files
files.download("vectors.tsv")
files.download("metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## RECURRENT NEURAL NETWORK (RNN)
Useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.